# Textonyms
These are words that are spelled using [the same keys on a phone keypad](https://en.wikipedia.org/wiki/Predictive_text#Textonyms).

Let's create a way to load a particular keypad. The ```standard_keypad.txt``` file in this repo represents the ITU [E.161](https://en.wikipedia.org/wiki/E.161) recommended layout.

In [1]:
def load_keypad(file):
    """
    Loads the key:character mapping from a text file.
    Format should be key, then space, then all characters for that key.
    Input: text file.
    Output: translation table produced by str.maketrans()
    """
    with open(file, 'r') as f:
        keys = [line.rstrip().lower() for line in f.readlines()]
    
    #create two lists for maketrans
    key_numbers = []
    key_letters = []
    
    for key in keys:
        if len(key) == 1: #that is, no letters assigned to it
            pass
        else:
            num, letts = key.split(' ')
            key_numbers.extend(num * len(letts))
            key_letters.extend(letts)
            
    table = str.maketrans(''.join(key_letters), ''.join(key_numbers))
        
    return table

In [2]:
def keypresses(string, table):
    """
    Make a textonym from a string and a lookup table.
    """
    return str.translate(string.lower(), table)

Let's try it out:

In [3]:
keypad = load_keypad('standard_keypad.txt')
keypresses('good', keypad)

'4663'

That works as it should. Now we'll create a dictionary of English words and their textonyms against which a user can look up a word of their choice. There are lots of word lists out there but ```popular.txt``` from [https://github.com/dolph/dictionary/blob/master/popular.txt] seems a good size, with 26,000 entries.

In [4]:
def create_dictionary(file, table):
    """
    Create a dictionary of word:textonym pairs.
    Input: line-delimited text file of words, plus table
    from load_keypad().
    Output: dictionary.
    """
    with open(file, 'r') as f:
        words = [line.rstrip() for line in f.readlines()]
    
    textonyms = [keypresses(word, table) for word in words]
    dictionary = dict(zip(words, textonyms))
    
    return dictionary

In [5]:
popular_words = create_dictionary('popular.txt', keypad)

Finally, we make a function that returns a list of textonyms for a given string. If the word provided by the user is also in the big word list, it will return a match against itself, which we probably don't want. We strip out these self-matches before returning the result.

In [6]:
def textonyms(input_word, dictionary, table):
    """
    For a given word, find possible textonyms.
    Input: string, plus dictionary from create_dictionary(),
    plus table from load_keypad()
    Output: list of matching strings.
    """
    matches = [word for word,textonym in dictionary.items() if textonym == keypresses(input_word, table)]
    matches = [word for word in matches if word != input_word] #remove self-match
    return matches

In [7]:
textonyms("test", popular_words, keypad)

['vest']